In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('data/Train_Test_datasets/Train_Test_Network_dataset/Train_Test_Network.csv', skipinitialspace=True)

df.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [3]:
df.drop(columns=['ts', 'src_ip', 'src_port', 'dst_ip', 'dst_port'], inplace=True)

# for now, only work with binary classification. ignore attack type, 'type'
df.drop(columns=['type'], inplace=True)

In [4]:
df.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,0,-,-,-,-,-,-,0
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,-,-,-,0
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,0


In [5]:
df.columns

Index(['proto', 'service', 'duration', 'src_bytes', 'dst_bytes', 'conn_state',
       'missed_bytes', 'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes',
       'dns_query', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD',
       'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed',
       'ssl_established', 'ssl_subject', 'ssl_issuer', 'http_trans_depth',
       'http_method', 'http_uri', 'http_version', 'http_request_body_len',
       'http_response_body_len', 'http_status_code', 'http_user_agent',
       'http_orig_mime_types', 'http_resp_mime_types', 'weird_name',
       'weird_addl', 'weird_notice', 'label'],
      dtype='object')

In [6]:
df.dtypes

proto                      object
service                    object
duration                  float64
src_bytes                   int64
dst_bytes                   int64
conn_state                 object
missed_bytes                int64
src_pkts                    int64
src_ip_bytes                int64
dst_pkts                    int64
dst_ip_bytes                int64
dns_query                  object
dns_qclass                  int64
dns_qtype                   int64
dns_rcode                   int64
dns_AA                     object
dns_RD                     object
dns_RA                     object
dns_rejected               object
ssl_version                object
ssl_cipher                 object
ssl_resumed                object
ssl_established            object
ssl_subject                object
ssl_issuer                 object
http_trans_depth           object
http_method                object
http_uri                   object
http_version               object
http_request_b

In [7]:
df.weird_name.value_counts()

-                                   459749
DNS_RR_unknown_type                    753
active_connection_reuse                275
data_before_established                109
bad_UDP_checksum                        68
bad_TCP_checksum                        52
connection_originator_SYN_ack           20
above_hole_data_without_any_acks         6
inappropriate_FIN                        3
dnp3_corrupt_header_checksum             3
possible_split_routing                   3
TCP_ack_underflow_or_misorder            2
Name: weird_name, dtype: int64

In [8]:
categorical_columns = [
    'proto',
    'service',
    'conn_state',
    'dns_AA',
    'dns_RD',
    'dns_RA',
    'dns_rejected',
    'dns_query',
    'ssl_version',
    'ssl_cipher',
    'ssl_resumed',
    'ssl_established',
    'ssl_subject',
    'ssl_issuer',
    'http_trans_depth',
    'http_method',
    'http_uri',
    'http_version',
    'http_user_agent',
    'http_orig_mime_types',
    'http_resp_mime_types',
    'weird_name',
    'weird_addl',
    'weird_notice',
]
numerical_columns = set(df.columns) - set(categorical_columns)
numerical_columns

{'dns_qclass',
 'dns_qtype',
 'dns_rcode',
 'dst_bytes',
 'dst_ip_bytes',
 'dst_pkts',
 'duration',
 'http_request_body_len',
 'http_response_body_len',
 'http_status_code',
 'label',
 'missed_bytes',
 'src_bytes',
 'src_ip_bytes',
 'src_pkts'}

In [9]:
# total_unique_values = 0
# for col in categorical_columns:
#     num_unique_values = df[col].nunique()
#     print(col, '(', num_unique_values, "unique values )", '\n', df[col].value_counts(), "\n")
#     total_unique_values += num_unique_values

# print(total_unique_values, "unique categorical values")

In [10]:
# drop the dns_query column for now - it has 14372 unique values, so we can't really one-hot encode it
df.drop(columns=['dns_query'], inplace=True)

In [11]:
categorical_columns = list(set(categorical_columns) - set(['dns_query']))
print(categorical_columns)

['ssl_resumed', 'http_orig_mime_types', 'ssl_established', 'http_resp_mime_types', 'weird_addl', 'http_user_agent', 'http_trans_depth', 'proto', 'ssl_issuer', 'dns_RA', 'conn_state', 'weird_name', 'ssl_subject', 'ssl_version', 'ssl_cipher', 'weird_notice', 'dns_RD', 'dns_AA', 'http_method', 'dns_rejected', 'service', 'http_uri', 'http_version']


In [12]:
pd.get_dummies(df, columns=categorical_columns)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_qclass,dns_qtype,...,http_uri_/ubuntu/dists/cosmic-security/InRelease,http_uri_/ubuntu/dists/cosmic-updates/InRelease,http_uri_/ubuntu/dists/cosmic/InRelease,http_uri_/ubuntu/pool/main/l/linux/linux-modules-4.18.0-17-generic_4.18.0-17.18_amd64.deb,http_uri_/upnp/control/ConnectionManager1,http_uri_/webmail/,http_uri_/webmail/src/login.php,http_uri_/webupd8team/java/ubuntu/dists/bionic/InRelease,http_version_-,http_version_1.1
0,80549.530260,1762852,41933215,0,252181,14911156,2,236,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.000000,0,0,0,1,63,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.000549,0,298,0,0,0,2,354,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,0.000000,0,0,0,1,60,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
461039,0.000000,0,0,0,0,0,1,103,0,0,...,0,0,0,0,0,0,0,0,1,0
461040,290.371539,101568,2592,0,108,108064,31,3832,0,0,...,0,0,0,0,0,0,0,0,1,0
461041,23.190902,32,31,0,8,411,7,395,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
df[list(numerical_columns)]

,src_pkts,http_request_body_len,src_bytes,duration,dst_pkts,dst_ip_bytes,src_ip_bytes,dns_qclass,missed_bytes,label,dst_bytes,http_response_body_len,dns_qtype,dns_rcode,http_status_code
0,252181,0,1762852,80549.530260,2,236,14911156,0,0,0,41933215,0,0,0,0
1,1,0,0,0.000000,0,0,63,0,0,0,0,0,0,0,0
2,0,0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0.000549,2,354,0,0,0,0,298,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,1,0,0,0.000000,0,0,60,0,0,1,0,0,0,0,0
461039,0,0,0,0.000000,1,103,0,0,0,1,0,0,0,0,0
461040,108,0,101568,290.371539,31,3832,108064,0,0,1,2592,0,0,0,0
461041,8,0,32,23.190902,7,395,411,0,0,1,31,0,0,0,0


In [14]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()

df_n = df

df_n.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,0,-,-,-,-,-,-,0
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,-,-,-,0
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,0


In [15]:
# df_n[list(numerical_columns)] = pd.DataFrame(standard_scaler.fit_transform(df[list(numerical_columns)]))
# df_n[list(numerical_columns)] = pd.DataFrame(robust_scaler.fit_transform(df[list(numerical_columns)]))
# df_n[list(numerical_columns)] = pd.DataFrame(min_max_scaler.fit_transform(df[list(numerical_columns)]))

In [16]:
df_n.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,0,-,-,-,-,-,-,0
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,0
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,-,-,-,0
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,0


In [19]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from time import time
from sklearn.model_selection import train_test_split


# try 85/15 and 90/10
# make a graph to show accuracies

y = df_n.label

# cats_partial = {
#                  # 'ssl_issuer', 'http_orig_mime_types', 'ssl_cipher', 'ssl_resumed', 'http_uri',
#                  'http_user_agent',
#                 'proto',
#                 'http_resp_mime_types',
#                 'dns_rejected',
#                 'http_method',
#                  'weird_name', 'dns_RD', 'dns_AA', 'ssl_subject', 'service', 'weird_notice', 'http_version',
#                  'dns_RA', 'weird_addl', 'ssl_version',
#     #             'http_trans_depth',
#     # 'ssl_established',
#     'conn_state',
# }
cats_partial = {
                 'ssl_issuer', 'http_orig_mime_types', 'ssl_cipher', 'ssl_resumed', 'http_uri',
                 'http_user_agent',
                'proto',
                'http_resp_mime_types',
                'dns_rejected',
                'http_method',
                 'weird_name', 'dns_RD', 'dns_AA', 'ssl_subject', 'service', 'weird_notice', 'http_version',
                 'dns_RA', 'weird_addl', 'ssl_version',
                'http_trans_depth',
    'ssl_established',
    'conn_state',
}

# df_n = pd.get_dummies(df, columns=categorical_columns)
x = df_n[list(cats_partial.union(numerical_columns).difference({'label'}))]
x = pd.get_dummies(x, columns=list(cats_partial))

tests = [.5, .2, .15, .1]
# test_size = .1
results = []

for test_size in tests:
    t1 = time()
    clf = MLPClassifier(
        hidden_layer_sizes=(200),
        # max_iter=5
        # random_state=3
    )

    # categorical columns only
    # x_categorical = x[list(set(x.columns) - numerical_columns)]

    # numeric columns only
    # x_numerical = x[list(numerical_columns - set(['label']))]

    x_train, x_test, y_train, y_test = train_test_split(
                                                        # x_numerical,
                                                        x,
                                                        y,
                                                        # random_state=2,
                                                        # stratify=y,
                                                        test_size=test_size
                                                       )
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    t2 = time()

    results.append({
        'test_size': test_size,
        'accuracy': accuracy_score(y_test, y_pred),
        'conf_matrix': confusion_matrix(y_test, y_pred),
        'time': t2 - t1,
        'f1_score': f1_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
    })


[{'test_size': 0.5, 'accuracy': 0.8724980695985632, 'conf_matrix': array([[125381,  24719],
       [  4673,  75749]]), 'time': 113.60527610778809, 'f1_score': 0.8375145115816242, 'precision': 0.7539614603654895, 'recall': 0.9418940091019871}, {'test_size': 0.2, 'accuracy': 0.9132080382609072, 'conf_matrix': array([[58230,  1776],
       [ 6227, 25976]]), 'time': 100.82517552375793, 'f1_score': 0.8665165540822284, 'precision': 0.936004612280196, 'recall': 0.8066329223985343}, {'test_size': 0.15, 'accuracy': 0.8432841216362769, 'conf_matrix': array([[39591,  5271],
       [ 5567, 18728]]), 'time': 102.9396140575409, 'f1_score': 0.7755828881434547, 'precision': 0.7803658485770241, 'recall': 0.7708582012759827}, {'test_size': 0.1, 'accuracy': 0.8943064743520226, 'conf_matrix': array([[28468,  1612],
       [ 3261, 12764]]), 'time': 138.69912385940552, 'f1_score': 0.8397092200914443, 'precision': 0.8878686700055648, 'recall': 0.7965054602184087}]
[{'test_size': 0.5, 'accuracy': 0.8724980695

In [22]:
from pprint import pprint
for r in results:
    print(r['test_size'])
    pprint(r)

0.5
{'accuracy': 0.8724980695985632,
 'conf_matrix': array([[125381,  24719],
       [  4673,  75749]]),
 'f1_score': 0.8375145115816242,
 'precision': 0.7539614603654895,
 'recall': 0.9418940091019871,
 'test_size': 0.5,
 'time': 113.60527610778809}
0.2
{'accuracy': 0.9132080382609072,
 'conf_matrix': array([[58230,  1776],
       [ 6227, 25976]]),
 'f1_score': 0.8665165540822284,
 'precision': 0.936004612280196,
 'recall': 0.8066329223985343,
 'test_size': 0.2,
 'time': 100.82517552375793}
0.15
{'accuracy': 0.8432841216362769,
 'conf_matrix': array([[39591,  5271],
       [ 5567, 18728]]),
 'f1_score': 0.7755828881434547,
 'precision': 0.7803658485770241,
 'recall': 0.7708582012759827,
 'test_size': 0.15,
 'time': 102.9396140575409}
0.1
{'accuracy': 0.8943064743520226,
 'conf_matrix': array([[28468,  1612],
       [ 3261, 12764]]),
 'f1_score': 0.8397092200914443,
 'precision': 0.8878686700055648,
 'recall': 0.7965054602184087,
 'test_size': 0.1,
 'time': 138.69912385940552}


In [ ]:
clf.get_params()

In [ ]:
sum(y_test)

In [ ]:
len(y_test)

In [ ]:
sum([52323,  7493, 5717, 26676])

In [ ]:
sum([5717, 26676])

In [ ]:
sum(y_pred)

In [ ]:
sum(y)

In [ ]:
print(x_test.columns)

In [ ]:
for c in x_train.columns:
    print(c)